In [1]:
import pymc3 as pm
import pandas as pd
import numpy as np
import theano.tensor as tt

from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation, Span, Label, Legend
from bokeh.io import output_notebook
from bokeh.palettes import brewer
output_notebook()

Loading BokehJS ...

In [7]:
data_monthly = pd.read_csv("../temp/SE_AUS_climate_TS/soilw.csv")
data_monthly.head(5)

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,0.653626,0.552399,0.507040,0.523567,0.555689,0.440926,0.420164,0.495448,0.164947,0.294967,...,0.000000,0.040149,0.00000,0.00000,0.000000,0.250712,0.298992,0.0,0.000000,0.209635
1,0.831187,0.755670,0.750689,0.734656,0.744165,0.661710,0.651576,0.738080,0.518147,0.598473,...,0.093473,0.000000,0.00000,0.00000,0.000000,0.128048,0.298862,0.0,0.000000,0.067914
2,0.915994,0.934604,0.842479,0.568775,0.644716,0.738585,0.828241,0.833308,0.247287,0.401972,...,0.080663,0.000000,0.04061,0.04857,0.239706,0.464354,0.499075,0.0,0.186728,0.357387
3,0.658562,0.774733,0.638003,0.331014,0.352817,0.387835,0.549325,0.481755,0.033791,0.170262,...,0.022939,0.000000,0.00000,0.00000,0.029833,0.282092,0.257404,0.0,0.000000,0.142161
4,0.156825,0.352856,0.228557,0.137998,0.115369,0.048011,0.197106,0.090299,0.000000,0.016474,...,0.146611,0.000000,0.00000,0.00000,0.084256,0.327575,0.322821,0.0,0.000000,0.140896


In [8]:
## normalize CO2 levels
y = data_monthly["0"].values
first_rec = y[0]
std_rec = np.std(y)
y_n = (y - first_rec) / std_rec
t = np.arange(len(y))

data_monthly = data_monthly.assign(t = t)
data_monthly = data_monthly.assign(y_n = y_n)

This data might be familiar to you, since it was used as an example in the Gaussian Processes for Machine Learning book by Rasmussen & Williams. The version of the data set they use starts in the late 1950’s, but stops at the end of 2003. So that our PyMC3 example is somewhat comparable to their example, we use the stretch of data from before 2004 as the “training” set. The data from 2004 to current we’ll use to test our predictions.

In [12]:

# make plot

p = figure(x_axis_type='datetime', title='Example',
           plot_width=850, plot_height=350)
p.yaxis.axis_label = 'Climate'
p.xaxis.axis_label = 'Date'

p.add_layout(ppm400)

p.line(data_monthly.index, data_monthly["0"],
       line_width=2, line_color="black", alpha=0.5)
p.circle(data_monthly.index, data_monthly["0"],
         line_color="black", alpha=0.1, size=2)

show(p)


## Modeling the Keeling Curve using GPs
As a starting point, we use the GP model described in Rasmussen & Williams. Instead of using flat priors on covariance function hyperparameters and then maximizing the marginal likelihood like is done in the textbook, we place somewhat informative priors on the hyperparameters and use optimization to find the MAP point. We use the gp.Marginal since Gaussian noise is assumed.

The R+W model is a sum of three GPs for the signal, and one GP for the noise.

1. A long term smooth rising trend represented by an exponentiated quadratic kernel.
2. A periodic term that decays away from exact periodicity. This is represented by the product of a Periodic covariance function and an exponentiated quadratic.
3. Small and medium term irregularities with a rational quadratic kernel.
4. The noise is modeled as the sum of an Exponential and a white noise kernel

The prior on CO2 as a function of time is,

The model in PyMC3¶
Below is the actual model. Each of the three component GPs is constructed separately. Since we are doing MAP, we use Marginal GPs and lastly call the .marginal_likelihood method to specify the marginal posterior.

In [23]:
# pull out normalized data
t = data_monthly["t"].values[:,None]
y = data_monthly["y_n"].values

In [24]:
with pm.Model() as model:
    # yearly periodic component x long term trend
    η_per = pm.HalfCauchy("η_per", beta=2, testval=1.0)
    ℓ_pdecay = pm.Gamma("ℓ_pdecay", alpha=10, beta=0.075)
    period  = pm.Normal("period", mu=1, sigma=0.05)
    ℓ_psmooth = pm.Gamma("ℓ_psmooth ", alpha=4, beta=3)
    cov_seasonal = η_per**2 * pm.gp.cov.Periodic(1, period, ℓ_psmooth) \
                            * pm.gp.cov.Matern52(1, ℓ_pdecay)
    gp_seasonal = pm.gp.Marginal(cov_func=cov_seasonal)

    # small/medium term irregularities
    η_med = pm.HalfCauchy("η_med", beta=0.5, testval=0.1)
    ℓ_med = pm.Gamma("ℓ_med", alpha=2, beta=0.75)
    α = pm.Gamma("α", alpha=5, beta=2)
    cov_medium = η_med**2 * pm.gp.cov.RatQuad(1, ℓ_med, α)
    gp_medium = pm.gp.Marginal(cov_func=cov_medium)

    # long term trend
    η_trend = pm.HalfCauchy("η_trend", beta=2, testval=2.0)
    ℓ_trend = pm.Gamma("ℓ_trend", alpha=4, beta=0.1)
    cov_trend = η_trend**2 * pm.gp.cov.ExpQuad(1, ℓ_trend)
    gp_trend = pm.gp.Marginal(cov_func=cov_trend)

    # noise model
    η_noise = pm.HalfNormal("η_noise", sigma=0.5, testval=0.05)
    ℓ_noise = pm.Gamma("ℓ_noise", alpha=2, beta=4)
    σ  = pm.HalfNormal("σ",  sigma=0.25, testval=0.05)
    cov_noise = η_noise**2 * pm.gp.cov.Matern32(1, ℓ_noise) +\
                pm.gp.cov.WhiteNoise(σ)

    # The Gaussian process is a sum of these three components
    gp = gp_seasonal + gp_medium + gp_trend

    # Since the normal noise model and the GP are conjugates, we use `Marginal` with the `.marginal_likelihood` method
    y_ = gp.marginal_likelihood("y", X=t, y=y, noise=cov_noise)

    # this line calls an optimizer to find the MAP
    mp = pm.find_MAP(include_transformed=True)

/users/global/doukel/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/users/global/doukel/anaconda3/lib/python3.7/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
/users/global/doukel/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` in

In [25]:
sorted([name+":"+str(mp[name]) for name in mp.keys() if not name.endswith("_")])

['period:1.0000000086298595',
 'α:2.1610322028035744',
 'η_med:1.0612796505573474',
 'η_noise:0.20059630151040034',
 'η_per:0.6438511008834682',
 'η_trend:0.006952158872063266',
 'σ:0.04258557715390243',
 'ℓ_med:6.488655709527093',
 'ℓ_noise:0.6283897491222303',
 'ℓ_pdecay:184.76480987630745',
 'ℓ_psmooth :0.9345664134191541',
 'ℓ_trend:29.028163511635004']

## Examining the fit of each of the additive GP components
The code below looks at the fit of the total GP, and each component individually. The total fit and its 2σ uncertainty is shown in red.

In [26]:
# predict at a 15 day granularity
dates = pd.date_range(start='3/15/1958', end="12/15/2003", freq="15D")
tnew = dates_to_idx(dates)[:,None]

print("Predicting with gp ...")
mu, var = gp.predict(tnew, point=mp, diag=True)
mean_pred = mu*std_co2 + first_co2
var_pred  = var*std_co2**2

# make dataframe to store fit results
fit = pd.DataFrame({"t": tnew.flatten(),
                    "mu_total": mean_pred,
                    "sd_total": np.sqrt(var_pred)},
                   index=dates)

print("Predicting with gp_trend ...")
mu, var = gp_trend.predict(tnew, point=mp,
                           given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                           diag=True)
fit = fit.assign(mu_trend = mu*std_co2 + first_co2,
                 sd_trend = np.sqrt(var*std_co2**2))

print("Predicting with gp_medium ...")
mu, var = gp_medium.predict(tnew, point=mp,
                            given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                            diag=True)
fit = fit.assign(mu_medium = mu*std_co2 + first_co2,
                 sd_medium = np.sqrt(var*std_co2**2))

print("Predicting with gp_seasonal ...")
mu, var = gp_seasonal.predict(tnew, point=mp,
                              given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                              diag=True)
fit = fit.assign(mu_seasonal = mu*std_co2 + first_co2,
                 sd_seasonal = np.sqrt(var*std_co2**2))
print("Done")

NameError: name 'dates_to_idx' is not defined

In [12]:
## plot the components
p = figure(title="Decomposition of the Mauna Loa Data",
           x_axis_type='datetime', plot_width=550, plot_height=350)
p.yaxis.axis_label = 'CO2 [ppm]'
p.xaxis.axis_label = 'Date'

# plot mean and 2σ region of total prediction
upper = fit.mu_total + 2*fit.sd_total
lower = fit.mu_total - 2*fit.sd_total
band_x = np.append(fit.index.values, fit.index.values[::-1])
band_y = np.append(lower, upper[::-1])

# total fit
p.line(fit.index, fit.mu_total,
       line_width=1, line_color="firebrick", legend="Total fit")
p.patch(band_x, band_y,
        color="firebrick", alpha=0.6, line_color="white")

# trend
p.line(fit.index, fit.mu_trend,
       line_width=1, line_color="blue", legend="Long term trend")

# medium
p.line(fit.index, fit.mu_medium,
       line_width=1, line_color="green", legend="Medium range variation")

# seasonal
p.line(fit.index, fit.mu_seasonal,
       line_width=1, line_color="orange", legend="Seasonal process")

# true value
p.circle(data_early.index, data_early['CO2'], size = 2,
         color="black", legend="Observed data")
p.legend.location = "top_left"
show(p)

The fit matches the observed data very well. The trend, seasonality, and short/medium term effects also are cleanly separated out. If you zoom so the seasonal process fills the plot window (from x equals 1955 to 2004, from y equals 310 to 320), it appears to be widening as time goes on. Lets plot the first year of each decade:

In [13]:
# plot several years

p = figure(title="Several years of the seasonal component",
           plot_width=550, plot_height=350)
p.yaxis.axis_label = 'Δ CO2 [ppm]'
p.xaxis.axis_label = 'Month'

colors = brewer['Paired'][5]
years = ["1960", "1970", "1980", "1990", "2000"]

for i, year in enumerate(years):
    dates = pd.date_range(start="1/1/"+year, end="12/31/"+year, freq="10D")
    tnew = dates_to_idx(dates)[:,None]

    print("Predicting year", year)
    mu, var = gp_seasonal.predict(tnew, point=mp, diag=True,
                                  given={"gp": gp, "X": t, "y": y, "noise": cov_noise})
    mu_pred = mu*std_co2

    # plot mean
    x = np.asarray((dates - dates[0])/pd.Timedelta(1, "M")) + 1
    p.line(x, mu_pred,
           line_width=1, line_color=colors[i], legend=year)

p.legend.location = "bottom_left"
show(p)

Predicting year 1960


/users/global/doukel/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x


Predicting year 1970
Predicting year 1980
Predicting year 1990
Predicting year 2000


This plot makes it clear that there is a broadening over time. So it would seem that as there is more CO2 in the atmosphere, the absorbtion/release cycle due to the growth and decay of vegetation in the northern hemisphere becomes more slightly more pronounced.

## What day will the CO2 level break 400 ppm?¶
How well do our forecasts look? Clearly the observed data trends up and the seasonal effect is very pronounced. Does our GP model capture this well enough to make reasonable extrapolations? Our “training” set went up until the end of 2003, so we are going to predict from January 2014 out to the end of 2017.

Although there isn’t any particular significance to this event other than it being a nice round number, our side goal was to see how well we could predict the date when the 400 ppm mark is first crossed. This event first occurred during May, 2013 and there were a few news articles about other significant milestones.

In [14]:
dates = pd.date_range(start="11/15/2003", end="12/15/2020", freq="10D")
tnew = dates_to_idx(dates)[:,None]

print("Sampling gp predictions ...")
mu_pred, cov_pred = gp.predict(tnew, point=mp)

# draw samples, and rescale
n_samples = 2000
samples = pm.MvNormal.dist(mu=mu_pred, cov=cov_pred).random(size=n_samples)
samples = samples * std_co2 + first_co2

Sampling gp predictions ...


/users/global/doukel/anaconda3/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x


In [15]:
### make plot
p = figure(x_axis_type='datetime', plot_width=700, plot_height=300)
p.yaxis.axis_label = 'CO2 [ppm]'
p.xaxis.axis_label = 'Date'

### plot mean and 2σ region of total prediction
# scale mean and var
mu_pred_sc = mu_pred * std_co2 + first_co2
sd_pred_sc = np.sqrt(np.diag(cov_pred) * std_co2**2 )

upper = mu_pred_sc + 2*sd_pred_sc
lower = mu_pred_sc - 2*sd_pred_sc
band_x = np.append(dates, dates[::-1])
band_y = np.append(lower, upper[::-1])

p.line(dates, mu_pred_sc,
       line_width=2, line_color="firebrick", legend="Total fit")
p.patch(band_x, band_y,
        color="firebrick", alpha=0.6, line_color="white")

# some predictions
idx = np.random.randint(0, samples.shape[0], 10)
p.multi_line([dates]*len(idx), [samples[i,:] for i in idx],
             color="firebrick", alpha=0.5, line_width=0.5)
# true value
#p.line(data_later.index, data_later['CO2'],
#       line_width=2, line_color="black", legend="Observed data")
p.circle(data_later.index, data_later['CO2'],
         color="black", legend="Observed data")

ppm400 = Span(location=400,
              dimension='width', line_color='black',
              line_dash='dashed', line_width=1)
p.add_layout(ppm400)
p.legend.location = "bottom_right"
show(p)

The mean prediction and the 2σ uncertainty is in red. A couple samples from the marginal posterior are also shown on there. It looks like our model was a little optimistic about how much CO2 is being released. The first time the 2σ uncertainty crosses the 400 ppm threshold is in May 2015, two years late.

One reason this is occuring is because our GP prior had zero mean. This means we encoded prior information that says that the function should go to zero as we move away from our observed data. This assumption probably isn’t justified. It’s also possible that the CO2 trend is increasing faster than linearly  important knowledge for accurate predictions. Another possibility is the MAP estimate. Without looking at the full posterior, the uncertainty in our estimates is underestimated. How badly is unknown.

Having a zero mean GP prior is causing the prediction to be pretty far off. Some possibilities for fixing this is to use a constant mean function, whose value could maybe be assigned the historical, or pre-industrial revolution, CO2 average. This may not be the best indicator for future CO2 levels though.

Also, using only historical CO2 data may not be the best predictor. In addition to looking at the underlying behavior of what determines CO2 levels using a GP fit, we could also incorporate other information, such as the amount of CO2 that is released by fossil fuel burning.

Next, we’ll see about using PyMC3’s GP functionality to improve the model, look at full posteriors, and incorporate other sources of data on drivers of CO2 levels.